In [ ]:
%%capture
# Cài đặt Unsloth và Transformers
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.55.4 # Phải khớp lúc train
!pip install --no-deps trl==0.22.2
!pip install sacrebleu
!pip install unbabel-comet

In [ ]:
import os
import torch
import sacrebleu
import numpy as np
import unicodedata
import re
import html
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer, SFTConfig

# --- CẤU HÌNH ĐƯỜNG DẪN & THAM SỐ CUỐI CÙNG ---
max_seq_length = 2048 # Giữ nguyên lúc train
dtype = None
load_in_4bit = True

TEST_EN_PATH = "/kaggle/input/d/ductoancau/vlsp-medical/MedicalDataset_VLSP/public_test.en.txt"
TEST_VI_PATH = "/kaggle/input/d/ductoancau/vlsp-medical/MedicalDataset_VLSP/public_test.vi.txt"
CHECKPOINT_PATH = "/kaggle/input/medical-training/outputs_adapter_en_vi/checkpoint-1000" # Checkpoint cuối cùng

BATCH_SIZE = 32 # Tối ưu hóa tốc độ Inference (Có thể thử 64 nếu T4 cho phép)

In [ ]:
#--- 1. LOAD MODEL VÀ ADAPTERS ---
print(f"⏳ Đang tải model từ: {CHECKPOINT_PATH}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = CHECKPOINT_PATH, 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map = "cuda:0",
)
FastLanguageModel.for_inference(model)
print("Model đã sẵn sàng cho Inference!")

In [ ]:
# --- 2. LOAD VÀ LÀM SẠCH DATA ---
def preprocess_text(text):
    if not isinstance(text, str): return ""
    text = html.unescape(text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'(?:https?://|www\.)\S+', '', text)
    text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)
    text = unicodedata.normalize('NFC', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print("Đang load và làm sạch Public Test...")
with open(TEST_EN_PATH, 'r', encoding='utf-8') as f: en_data = [preprocess_text(l) for l in f]
with open(TEST_VI_PATH, 'r', encoding='utf-8') as f: vi_data = [preprocess_text(l) for l in f]

# Reference phải là list of list cho sacrebleu
en_references = [[l.strip()] for l in en_data]
vi_references = [[l.strip()] for l in vi_data]
print(f"Đã load {len(en_data)} câu Test (sẵn sàng tính toán).")

In [ ]:
# #  PHASE 4: TÍNH BLEU SCORE CUỐI CÙNG (FINAL VERSION)
# # ==============================================================================
# import sacrebleu
# import torch

# sys_prompt_en_vi = (
#     "Bạn là một biên dịch viên y tế chuyên nghiệp. "
#     "Nhiệm vụ của bạn là dịch chính xác văn bản y khoa từ tiếng Anh sang tiếng Việt, "
#     "đảm bảo văn phong khoa học và thuật ngữ chính xác."
# )
# sys_prompt_vi_en = (
#     "You are a professional medical translator. "
#     "Your task is to accurately translate the following Vietnamese medical text into English. "
#     "Ensure correct medical terminology and academic style."
# )

# # 1. QUAN TRỌNG NHẤT: Đổi padding sang trái cho tác vụ sinh văn bản
# tokenizer.padding_side = "left" 

# def batch_translate_and_score(source_texts, target_references, direction, batch_size=BATCH_SIZE):
#     sys_prompt = sys_prompt_en_vi if direction == "en_vi" else sys_prompt_vi_en
#     hypotheses = []
#     model.eval()
    
#     print(f" Bắt đầu dịch {direction} (Batch size: {batch_size})...")
    
#     # Duyệt qua từng batch (Ví dụ: 16 câu một lần)
#     for i in range(0, len(source_texts), batch_size):
#         batch = source_texts[i : i + batch_size]
        
#         # 2. Tạo Prompt và Tokenize ngay trong vòng lặp (Tiết kiệm RAM)
#         batch_prompts = []
#         for text in batch:
#             messages = [{"role": "system", "content": sys_prompt}, {"role": "user", "content": text}]
#             prompt_str = tokenizer.apply_chat_template(
#                 messages,
#                 tokenize=False,
#                 add_generation_prompt=True,
#                 enable_thinking=False
#             )
#             batch_prompts.append(prompt_str)
            
#         # 3. Tokenize batch này và đẩy lên GPU ngay (Vì batch nhỏ nên an toàn)
#         inputs = tokenizer(
#             batch_prompts,
#             return_tensors="pt",
#             padding=True,
#             truncation=True,
#             max_length=max_seq_length
#         ).to("cuda")

#         # Lấy độ dài input dài nhất trong batch hiện tại
#         src_lens = inputs.attention_mask.sum(dim=1)
#         max_src_len = int(src_lens.max())
        
#         # 1. Tính độ dài output mong muốn (Hệ số 1.5 là hợp lý cho En->Vi)
#         desired_output_len = int(max_src_len * 1.5) + 50
        
#         # 2. Tính chỗ trống còn lại trong bộ nhớ (Buffer trừ hao 20 token cho an toàn)
#         max_allowed_len = max_seq_length - max_src_len - 20 
        
#         # 3. Chốt con số cuối cùng (Không được vượt quá chỗ trống còn lại)
#         DYNAMIC_MAX_NEW_TOKENS = min(desired_output_len, max_allowed_len)
        
#         # [QUAN TRỌNG] Đảm bảo tối thiểu vẫn sinh được gì đó (tránh ra số âm nếu input quá dài)
#         DYNAMIC_MAX_NEW_TOKENS = max(DYNAMIC_MAX_NEW_TOKENS, 100)

#         # 4. Sinh văn bản
#         with torch.no_grad():
#             outputs = model.generate(
#                 input_ids = inputs.input_ids,
#                 attention_mask = inputs.attention_mask,
#                 max_new_tokens = DYNAMIC_MAX_NEW_TOKENS,
#                 # repetition_penalty = 1.2,
#                 # length_penalty = 1.0,
#                 # no_repeat_ngram_size = 3,
#                 do_sample = False,
#                 num_beams = 1,
#                 use_cache = True,
#                 pad_token_id = tokenizer.pad_token_id,
#                 eos_token_id = tokenizer.eos_token_id 
#             )
        
#         # 5. Cắt bỏ phần prompt khỏi kết quả (Slicing chuẩn xác)
#         # Lấy độ dài thực tế của input (bao gồm padding bên trái)
#         input_len_total = inputs.input_ids.shape[1]
        
#         for output_seq in outputs:
#             generated_tokens = output_seq[input_len_total:] # Cắt sạch phần prompt
#             decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
#             hypotheses.append(decoded)
            
#         print(f"   -> Đã dịch {direction}: {min(i + batch_size, len(source_texts))}/{len(source_texts)} câu...", end='\r')
        
#     # 6. Tính BLEU (Format chuẩn cho sacrebleu: list of lists)
#     # Target references phải được bọc trong 1 list nữa
#     bleu = sacrebleu.corpus_bleu(hypotheses, [target_references])
    
#     return bleu.score, hypotheses


# print("\n" + "="*60)
# print(f" BẮT ĐẦU TÍNH BLEU SCORE (EN-VI SPECIALIST)")
# print("="*60)

# # Chạy test
# # valid_en: List các câu tiếng Anh
# # vi_references: List các câu đáp án tiếng Việt (dạng string thuần túy: ['câu 1', 'câu 2'])
# bleu_en_vi, hypotheses_en_vi = batch_translate_and_score(en_data, vi_data, "en_vi", batch_size=BATCH_SIZE)
# print(f"\n\n KẾT QUẢ BLEU SCORE (ANH -> VIỆT): {bleu_en_vi:.2f}")

In [ ]:
# ==============================================================================
# HÀM DỊCH & TÍNH ĐIỂM EN-VI (FULL: DỊCH + METRICS + NLTK FIX)
# ==============================================================================
import sacrebleu
from comet import download_model, load_from_checkpoint
import torch
import os
import re
import nltk
from nltk.translate.meteor_score import meteor_score
from tqdm import tqdm

# --- 1. SETUP NLTK CHO METEOR (Chạy 1 lần) ---
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/wordnet.zip')
    nltk.data.find('corpora/omw-1.4.zip')
except LookupError:
    print("⬇ Đang tải dữ liệu NLTK...")
    nltk.download('punkt', quiet=True)
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)

# --- 2. SYSTEM PROMPT (EN -> VI) ---
sys_prompt_en_vi = (
    "Bạn là một biên dịch viên y tế chuyên nghiệp. "
    "Nhiệm vụ của bạn là dịch chính xác văn bản y khoa từ tiếng Anh sang tiếng Việt, "
    "đảm bảo văn phong khoa học và thuật ngữ chính xác."
)

def run_pipeline_en_vi(src_list, ref_list, output_filename="hypotheses_en_vi.txt"):
    """
    src_list: List các câu tiếng Anh (en_data)
    ref_list: List các câu đáp án tiếng Việt (vi_data)
    """
    
    # ==========================================================================
    # PHẦN A: DỊCH THUẬT (GENERATION)
    # ==========================================================================
    if os.path.exists(output_filename):
        os.remove(output_filename)
        
    batch_size = 32 # Chạy max tốc độ
    print(f" Đang dịch {len(src_list)} câu (Anh -> Việt) | Batch Size: {batch_size}...", flush=True)
    
    hypotheses = []
    
    for i in range(0, len(src_list), batch_size):
        batch = src_list[i : i + batch_size]
        
        # Tạo Prompt
        batch_prompts = [
            tokenizer.apply_chat_template(
                [{"role": "system", "content": sys_prompt_en_vi}, 
                 {"role": "user", "content": t}],
                tokenize=False, add_generation_prompt=True, enable_thinking=False
            ) for t in batch
        ]
        
        # Tokenize
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=2048).to("cuda")
        src_lens = inputs.attention_mask.sum(dim=1)
        max_new = min(int(src_lens.max() * 1.5) + 50, 2048)
        
        # Generate
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs.input_ids, attention_mask=inputs.attention_mask,
                max_new_tokens=max_new,
                do_sample=False, num_beams=1, repetition_penalty=1.0, use_cache=True,
                pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id
            )
            
        # Decode & Clean
        batch_hyps = []
        input_len = inputs.input_ids.shape[1]
        for seq in outputs:
            decoded = tokenizer.decode(seq[input_len:], skip_special_tokens=True).strip()
            decoded = re.sub(r'<think>.*?</think>', '', decoded, flags=re.DOTALL).strip()
            batch_hyps.append(decoded)
            
        # Lưu file ngay lập tức
        with open(output_filename, "a", encoding="utf-8") as f:
            for h in batch_hyps: f.write(h.replace('\n', ' ') + '\n')
            
        hypotheses.extend(batch_hyps)
        print(f"   -> Progress: {min(i+batch_size, len(src_list))}/{len(src_list)}...", end='\r', flush=True)

    print(f"\n Dịch xong {len(hypotheses)} câu. Đã lưu vào {output_filename}", flush=True)

    # ==========================================================================
    # PHẦN B: TÍNH TOÁN CHỈ SỐ (METRICS)
    # ==========================================================================
    print("\n" + "="*50)
    print("KẾT QUẢ ĐÁNH GIÁ (EN -> VI)")
    print("="*50)
    
    # 1. TER (SacreBLEU)
    score_ter = sacrebleu.corpus_ter(hypotheses, [ref_list])
    print(f"TER Score:    {score_ter.score:.2f} (📉 Càng thấp càng tốt)", flush=True)

    # 2. METEOR (NLTK - Không lo lỗi Java)
    print(" Đang tính METEOR (NLTK)...", flush=True)
    meteor_scores = []
    for hyp, ref in tqdm(zip(hypotheses, ref_list), total=len(hypotheses), desc="METEOR"):
        hyp_tokens = nltk.word_tokenize(hyp)
        ref_tokens = nltk.word_tokenize(ref)
        score = meteor_score([ref_tokens], hyp_tokens)
        meteor_scores.append(score)
    avg_meteor = (sum(meteor_scores) / len(meteor_scores)) * 100
    print(f"METEOR Score: {avg_meteor:.2f} (Càng cao càng tốt)", flush=True)

    # 3. CHRF++ (SacreBLEU)
    score_chrf = sacrebleu.corpus_chrf(hypotheses, [ref_list], word_order=2)
    print(f"CHRF++ Score: {score_chrf.score:.2f} (Càng cao càng tốt)", flush=True)

    # 4. COMET (Unbabel)
    print("\n Đang tính COMET...", flush=True)
    try:
        model_path = download_model("Unbabel/wmt22-comet-da")
        comet_model = load_from_checkpoint(model_path)
        data = [{"src": s, "mt": h, "ref": r} for s, h, r in zip(src_list, hypotheses, ref_list)]
        score_comet = comet_model.predict(data, batch_size=16, gpus=1)
        print(f" COMET Score:  {score_comet.system_score:.4f} (Càng cao càng tốt)", flush=True)
    except Exception as e:
        print(f" Lỗi COMET: {e}")

# ==============================================================================
#  CHẠY THÔI (EN -> VI)
# ==============================================================================
# Input: Tiếng Anh (en_data) | Đáp án: Tiếng Việt (vi_data)
run_pipeline_en_vi(en_data, vi_data)